In [1]:
import pandas as pd
import json
import numpy as np
from ast import literal_eval
from datetime import datetime, timedelta
import re
from rapidfuzz import fuzz
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import simplemma
from nltk.stem.snowball import SnowballStemmer

pd.set_option("max_colwidth", None)
pd.set_option("display.max_columns", None)

In [2]:
path_data = "data_mass/"

In [4]:
path_cv = f"{path_data}slesar_santehnik.json"
with open(path_cv, "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
path_cv = f"{path_data}electromontajnik.json"
with open(path_cv, "r", encoding="utf-8") as f:
    data_1 = json.load(f)

In [6]:
path_cv = f"{path_data}injener_energetik.json"
with open(path_cv, "r",  encoding="utf-8") as f:
    data_2 = json.load(f)

In [7]:
path_cv = f"{path_data}svarschik.json"
with open(path_cv, "r",  encoding="utf-8") as f:
    data_3 = json.load(f)

In [ ]:
path_cv = f"{path_data}svarschik.json"
with open(path_cv, "r",  encoding="utf-8") as f:
    data_3 = json.load(f)

In [10]:
data_1_ = []
for x in data_1:
    try:
        _ = x.keys()
        data_1_.append(x)
    except Exception as e:
        print(e)
        print("Error in data_1:", x)
        pass

data_2_ = []
for x in data_2:
    try:
        _ = x.keys()
        data_2_.append(x)
    except Exception as e:
        print(e)
        print("Error in data_2:", x)
        pass

data_3_ = []
for x in data_3:
    try:
        _ = x.keys()
        data_3_.append(x)
    except Exception as e:
        print(e)
        print("Error in data_3:", x)
        pass

In [11]:
len(data)

2076

In [22]:
df_init_0 = pd.DataFrame(data)
df_init_1 = pd.DataFrame(data_1_)
df_init_2 = pd.DataFrame(data_2_)
df_init_3 = pd.DataFrame(data_3_)

# df_init = pd.concat([df_init, df_init_1, df_init_2])

In [23]:
df_init_0.head(2)

,source,link,title,salary,phone,address,description,photoUrl,mainParams,additionalParams,techInfo
0,avito,/kigoma/rezume/slesar_santehnik_7355287864?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJaTHRQaUI2dTJDTkpvVTRUIjt9OvI6Gj8AAAA,None,,,Восточная провинция,,https://10.img.avito.st/image/1/1.MsUqrba4niwcBFwpQt1C71gMnCqUDBwkXAmcLpoEliac.yjXtf_lO3_jXYScF733U-_l0o0syq97xZeH6QQ3VFqE,"{'Тип занятости': 'Основная работа', 'Может выйти завтра': 'Да', 'Стаж работы': 'Без опыта', 'Пол': 'Мужской', 'Возраст': '27 лет', 'Сфера деятельности': 'ЖКХ, эксплуатация', 'График работы': 'Свободный график'}","{'Гражданство': ['Гражданство — Таджикистан Разрешение на работу в России — Нет, Гражданство — Таджикистан, Разрешение на работу в России — Нет']}","{'itemId': '№ 7355287864', 'itemDate': '· вчера в 12:34', 'totalViews': '2 просмотра', 'todayViews': ' (+1 сегодня)'}"
1,avito,/domodedovo/rezume/slesar_santehnik_7321175653?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJaTHRQaUI2dTJDTkpvVTRUIjt9OvI6Gj8AAAA,Главная,,,"Московская обл., Домодедово, мкр-н Востряково, Донская ул., 5",Работал по частным домам и квартирам,no photo,"{'Тип занятости': 'Основная работа', 'Готовность к командировкам': 'Иногда', 'Может выйти завтра': 'Да', 'Образование': 'Среднее специальное', 'Стаж работы': '4 года', 'Переезд': 'Невозможен', 'Пол': 'Мужской', 'Возраст': '46 лет', 'Сфера деятельности': 'ЖКХ, эксплуатация', 'График работы': 'Полный день'}","{'Учебные заведения': ['спту-12 Слесарь сантехник, спту-12, Слесарь сантехник'], 'Гражданство': ['Гражданство — Россия, Гражданство — Россия'], 'Опыт работы': ['4 года и 11 месяцев, Июнь 1994 — Апрель 1999, Жкх, Сантехник ремонт и замена сантехнических узлов, Жкх, Сантехник, ремонт и замена сантехнических узлов']}","{'itemId': '№ 7321175653', 'itemDate': '· вчера в 22:09', 'totalViews': '1 просмотр', 'todayViews': ' (+0 сегодня)'}"


In [24]:
dfs = []
for df_init, label in zip([df_init_0, df_init_1, df_init_2, df_init_3], ["Слесарь-сантехник", "Электромонтажник", "Инженер-энергетик", "Сварщик"]):
    df_init = df_init.dropna()
    
    df_main_params = pd.DataFrame(df_init["mainParams"].to_list())
    df_add_params = pd.DataFrame(df_init["additionalParams"].to_list())
    df_techInfo_params = pd.DataFrame(df_init["techInfo"].to_list())
    
    df_full = pd.concat([df_init.drop(columns = ["mainParams", "additionalParams", "techInfo"]), df_main_params, df_add_params, df_techInfo_params], axis=1)
    df_full["Должность"] = label
    dfs.append(df_full)
    print(df_full.shape)

(377, 32)
(1429, 32)
(978, 33)
(2145, 32)


In [25]:
df_full = pd.concat(dfs)

In [27]:
def prep_list_feats(feat_list, feat_name):
    if type(feat_list) == float:
        return "Нет данных"
    final_list = []
    for x in feat_list[0].split(", "):
        final_list.extend(x.split("\n"))
    result = ", ".join(list(set(final_list))).replace(f"{feat} — ", "")
    if feat == "Категория прав":
        result = result.strip().replace(" ", ", ")
    return result

In [28]:
feats_list_prep = ["Свой транспорт", "Стаж вождения", "Гражданство", "Водительское удостоверение", "Категория прав", "Опыт работы", "Знание языков", "Учебные заведения", "Медкнижка"]
for feat in feats_list_prep:
    df_full[feat] = df_full[feat].apply(lambda x: prep_list_feats(x, feat))

In [29]:
df_full = df_full.fillna("Нет данных").replace("", "Нет данных").replace("no photo", "Нет данных")

In [30]:
df_full.head(3)

,source,link,title,salary,phone,address,description,photoUrl,Тип занятости,Готовность к командировкам,Может выйти завтра,Образование,Стаж работы,Переезд,Пол,Возраст,Сфера деятельности,График работы,Учебные заведения,Гражданство,Опыт работы,Знание языков,Медкнижка,Водительское удостоверение,Свой транспорт,Стаж вождения,Категория прав,itemId,itemDate,totalViews,todayViews,Должность,Дата осмотра терапевтом
1,avito,/domodedovo/rezume/slesar_santehnik_7321175653?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJaTHRQaUI2dTJDTkpvVTRUIjt9OvI6Gj8AAAA,Главная,Нет данных,Нет данных,"Московская обл., Домодедово, мкр-н Востряково, Донская ул., 5",Работал по частным домам и квартирам,Нет данных,Основная работа,Нет данных,Нет данных,Среднее,1 год,Нет данных,Мужской,20 лет,"ЖКХ, эксплуатация",Свободный график,Нет данных,Россия,"Спайка труб, Жкх, Апрель 2023 — Апрель 2024, 1 год и 1 месяц, Сантехник, установка сантехники",Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,№ 7307964822,· сегодня в 08:02,1 просмотр,(+1 сегодня),Слесарь-сантехник,Нет данных
2,avito,/sankt-peterburg_peterhof/rezume/slesar_santehnik_7307964822?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJaTHRQaUI2dTJDTkpvVTRUIjt9OvI6Gj8AAAA,Главная,Нет данных,Показать телефон\n8 994 XXX-XX-XX,"Санкт-Петербург, Петродворцовый р-н, Петергоф, Ропшинское ш., 3к7\nр-н Петродворцовый",Нет данных,Нет данных,Основная работа,Нет данных,Да,Нет данных,Без опыта,Нет данных,Мужской,38 лет,"ЖКХ, эксплуатация",Полный день,Нет данных,Россия,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,№ 7338624844,· сегодня в 08:12,1 просмотр,(+1 сегодня),Слесарь-сантехник,Нет данных
3,avito,/domodedovo/rezume/slesar_santehnik_7338624844?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJaTHRQaUI2dTJDTkpvVTRUIjt9OvI6Gj8AAAA,Главная,Нет данных,Показать телефон\n8 915 XXX-XX-XX,"Московская область, Домодедово, микрорайон Южный, ЖК Новое Домодедово",Нет данных,Нет данных,Подработка,Не готов,Нет данных,Среднее специальное,10 лет,Невозможен,Мужской,28 лет,"ЖКХ, эксплуатация",Свободный график,"Колледж Водных Ресурсов, Монтажник Санитарно-технических систем и оборудования",Россия,"замени унитазов, Монтаж канализации, замена смесителей, Слесарь ремонтник, монтаж водопровода., ГУП ВОДОКАНАЛ САНКТ-ПЕТЕРБУРГА, 10 лет и 3 месяца, Февраль 2015 — настоящее время",Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,№ 7218828875,· вчера в 11:00,159 просмотров,(+1 сегодня),Слесарь-сантехник,Нет данных


In [31]:
df_full["Переезд"].value_counts()

Переезд
Нет данных    3041
Невозможен    1273
Возможен       615
Name: count, dtype: int64

In [32]:
df_full.loc[df_full["Переезд"] == "Да", "Переезд"] = "Возможен"
df_full.loc[df_full["Переезд"] == "Нет", "Переезд"] = "Невозможен"

In [33]:
df_full["Категория прав"].value_counts()

Категория прав
Нет данных                   4558
B                             224
B, C                           58
A, B                           35
A, B, C                        19
B, M                           11
B, BE                           4
A, B, BE, C                     3
B, C, D                         3
B, C, CE, D                     2
A, B, BE, C, M                  1
A, B, C, M                      1
A, B, M                         1
B, C, M                         1
B, C, D, M                      1
C, CE                           1
A, B, BE, C, CE, D, DE, M       1
B, D                            1
B, C, CE                        1
B, BE, C                        1
A, B, C, CE, M                  1
B, BE, C, CE                    1
Name: count, dtype: int64

In [34]:
df_full["salary"].value_counts()

salary
Нет данных    4929
Name: count, dtype: int64

## Match location

In [36]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.location import Location
from time import sleep
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_fixed

tqdm.pandas()

In [37]:
from request_api import process_corpus

In [630]:
df_vac = pd.read_excel(f"{path_data}Вакансии демо.xlsx", sheet_name="Вакансия 2").fillna("Нет данных")

In [631]:
df_vac.columns = [x.strip() for x in df_vac.columns]

In [632]:
df_vac.loc[0, "Должность"] = 'Горнорабочий очистного забоя (ученик)'

In [544]:
address_vac = df_vac.iloc[0]["Адрес"]
df_full["address"] = df_full["address"].apply(lambda x: x.split("\n")[0])

In [545]:
address_vac

'город Североуральск, Свердловская область'

Расстояние Ливенштейна подходит плохо, потому что путается Красноярский и Краснодарский край. Попробуем bm25, основанный на tf-idf

In [38]:
geolocator = Nominatim(user_agent="uniweb_100")

In [39]:
@retry(stop=stop_after_attempt(3), wait=wait_fixed(1.1))
def get_distance(data: str):
    coords_ref_str, adress_search = data.split("[SEP]")
    coords_ref = literal_eval(coords_ref_str)
    loc_search = geolocator.geocode(adress_search, language="ru")
    if loc_search is None:
        return float("nan"), adress_search
    coords_search = loc_search.latitude, loc_search.longitude
    distance = geodesic(coords_ref, coords_search).kilometers
    return distance, loc_search.address

In [40]:
@retry(stop=stop_after_attempt(3), wait=wait_fixed(1.1))
def get_coords(adress_search: str):
    loc_search = geolocator.geocode(adress_search, language="ru")
    if loc_search is None:
        return float("nan"), adress_search
    coords_search = loc_search.latitude, loc_search.longitude
    return coords_search, loc_search.address

In [41]:
def shorten_address(address: str):
    split_ = address.split(", ")
    split_ = split_[:3]
    result = []
    for item in split_:
        norm = True
        key_words = ["ул.", "улиц", "пер.", "переулок", "пл.", "площ"]
        for key in key_words:
            if key in item:
                norm = False
                break
        if norm:
            result.append(item)
    return ", ".join(result)

In [ ]:
address_vac = "Россия, Красноярский край, Ирбейское"

In [43]:
location_vac = geolocator.geocode(address_vac, language="ru")
location_vac

Location(Ирбейское, Ирбейский сельсовет, Ирбейский район, Красноярский край, Сибирский федеральный округ, Россия, (55.644501, 95.456528, 0.0))

In [142]:
coords_vac = location_vac.latitude, location_vac.longitude
print(location_vac.address)

AttributeError: 'NoneType' object has no attribute 'latitude'

In [44]:
df_full["address_short"] = df_full["address"].apply(shorten_address)

In [47]:
dfs = []
for i in range(150, df_full.shape[0], 150):
    geolocator = Nominatim(user_agent=f"uniweb_{i+1}")
    df_subset = df_full.iloc[i:i+150]
    coords_list = []
    address_full_list = []
    for x in tqdm(df_subset["address_short"].to_list()):
        try:
            coords, address_full = get_coords(x)
        except Exception:
            coords = None
            address_full = x
        coords_list.append(coords)
        address_full_list.append(address_full)
    df_subset['coords'] = coords_list
    df_subset["address_full"] = address_full_list
    df_subset.to_csv(f"./tmp_candidates_3/candidates_tmp_{i}.csv", index = False)
    dfs.append(df_subset)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [04:38<00:00,  1.85s/it]
/var/folders/1k/4qnh82lj5f16195540wwdb5h0000gn/T/ipykernel_469/2905491127.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['coords'] = coords_list
/var/folders/1k/4qnh82lj5f16195540wwdb5h0000gn/T/ipykernel_469/2905491127.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [48]:
df_full__  = pd.concat(dfs)
df_full__.shape

(4779, 36)

In [49]:
df_full__.to_csv(f"./data_jobs/candidates_tmp_3.csv", index = False)

In [51]:
df_full__["coords"].isna().sum()

2406

In [52]:
df_full = df_full__.dropna(subset=["coords"])

In [53]:
tmp = df_full["Переезд"].values

## Prepared df

In [634]:
df_full = pd.read_csv(f"./data_jobs/Горнорабочий очистного забоя (ученик).csv")

In [635]:
df_full["Переезд"] = tmp

In [54]:
df_full.head(2)

,source,link,title,salary,phone,address,description,photoUrl,Тип занятости,Готовность к командировкам,Может выйти завтра,Образование,Стаж работы,Переезд,Пол,Возраст,Сфера деятельности,График работы,Учебные заведения,Гражданство,Опыт работы,Знание языков,Медкнижка,Водительское удостоверение,Свой транспорт,Стаж вождения,Категория прав,itemId,itemDate,totalViews,todayViews,Должность,Дата осмотра терапевтом,address_short,coords,address_full
153,avito,/baykit/rezume/slesar_santehnik_7259471769?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJYc3VwMnpSeEE5MWE5dmhyIjt9R4egVz8AAAA,Главная,Нет данных,Нет данных,"Красноярский край, Эвенкийский район","спокойствие, обучаемость,",https://40.img.avito.st/image/1/1.1b2QI7a4eVSmirtRtjSUtZuCe1Iugvtc5od7ViCKcV4m.bQThTSRexM8Pyg2J-IQj7LNVo5b4wvaJ6nYAcEH17Uk,Основная работа,Не готов,Да,Среднее,7 лет,Невозможен,Мужской,35 лет,"ЖКХ, эксплуатация",Полный день,Нет данных,Россия,"Мастер учястка, Проводил монтаж работы, Апрель 2018 — настоящее время, ООО ЭЛ-Техника, 7 лет и 1 месяц",Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,№ 7327731583,· 21 апреля в 14:23,4 просмотра,(+1 сегодня),Слесарь-сантехник,Нет данных,"Красноярский край, Эвенкийский район","(64.2695782, 99.5609121)","Эвенкийский район, Красноярский край, Сибирский федеральный округ, Россия"
158,avito,/krasnoyarsk/rezume/slesar_santehnik_7283925077?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJYc3VwMnpSeEE5MWE5dmhyIjt9R4egVz8AAAA,Главная,Нет данных,Нет данных,"Красноярский край, Красноярск, ул. Березина, 82В\nр-н Центральный",дипломированный слесарь сантехник 4 разряд водитель ВС,Нет данных,"Основная работа, Подработка",Иногда,Да,Высшее,4 года,Невозможен,Мужской,36 лет,Строительство,Сменный график,"Природообустройство и водопользование, брянский государственный аграрный университет",Россия,"Июль 2022 — Ноябрь 2023, 1 год и 5 месяцев, МУП ""Отрадное"", Слесарь сантехник, Сантехнические работы",Немецкий — начальный,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,№ 7323042571,· 19 апреля в 20:54,6 просмотров,(+2 сегодня),Слесарь-сантехник,Нет данных,"Красноярский край, Красноярск","(56.0091173, 92.872586)","Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, 660000, Россия"


In [55]:
feats_drop = ["phone", "photoUrl", "salary", "source", "title", "Пол", "Свой транспорт", "Знание языков", "Медкнижка", "todayViews", "totalViews"]
rename_columns = {
    "Сфера деятельности": "Компетенции",
    "Может выйти завтра": "Доступность",
    "description": "О себе",
    "itemId": "id"
}
feats_exp = ["Стаж работы", "Опыт работы"]
desc_columns = ["О себе", "Возраст", "Готовность к командировкам", "Гражданство"]
drive = ["Водительское удостоверение", "Категория прав", "Стаж вождения"]

In [56]:
df_full = df_full.drop(columns=feats_drop)
df_full = df_full.rename(columns=rename_columns)

In [57]:
new_labels = ["Опыт работы", "Описание", "Категория прав"]
feats_drop_2 = []
for label, feat_list in zip(new_labels, [feats_exp, desc_columns, drive]):
    df_full[label] = df_full[feat_list].apply(lambda x: "\n\n".join([f"{feat}:\n{value}" for feat, value in zip(feat_list, x)]), axis = 1)
    feats_drop_2 += feat_list

In [58]:
feats_drop_2 = list(set(feats_drop_2) - set(new_labels))

In [59]:
df_full = df_full.drop(columns=feats_drop_2)

In [60]:
df_full.head(2)

,link,address,Тип занятости,Доступность,Образование,Переезд,Компетенции,График работы,Учебные заведения,Опыт работы,Категория прав,id,itemDate,Должность,Дата осмотра терапевтом,address_short,coords,address_full,Описание
153,/baykit/rezume/slesar_santehnik_7259471769?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJYc3VwMnpSeEE5MWE5dmhyIjt9R4egVz8AAAA,"Красноярский край, Эвенкийский район",Основная работа,Да,Среднее,Невозможен,"ЖКХ, эксплуатация",Полный день,Нет данных,"Стаж работы:\n7 лет\n\nОпыт работы:\nМастер учястка, Проводил монтаж работы, Апрель 2018 — настоящее время, ООО ЭЛ-Техника, 7 лет и 1 месяц",Водительское удостоверение:\nНет данных\n\nКатегория прав:\nНет данных\n\nСтаж вождения:\nНет данных,№ 7327731583,· 21 апреля в 14:23,Слесарь-сантехник,Нет данных,"Красноярский край, Эвенкийский район","(64.2695782, 99.5609121)","Эвенкийский район, Красноярский край, Сибирский федеральный округ, Россия","О себе:\nспокойствие, обучаемость,\n\nВозраст:\n35 лет\n\nГотовность к командировкам:\nНе готов\n\nГражданство:\nРоссия"
158,/krasnoyarsk/rezume/slesar_santehnik_7283925077?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJYc3VwMnpSeEE5MWE5dmhyIjt9R4egVz8AAAA,"Красноярский край, Красноярск, ул. Березина, 82В\nр-н Центральный","Основная работа, Подработка",Да,Высшее,Невозможен,Строительство,Сменный график,"Природообустройство и водопользование, брянский государственный аграрный университет","Стаж работы:\n4 года\n\nОпыт работы:\nИюль 2022 — Ноябрь 2023, 1 год и 5 месяцев, МУП ""Отрадное"", Слесарь сантехник, Сантехнические работы",Водительское удостоверение:\nНет данных\n\nКатегория прав:\nНет данных\n\nСтаж вождения:\nНет данных,№ 7323042571,· 19 апреля в 20:54,Слесарь-сантехник,Нет данных,"Красноярский край, Красноярск","(56.0091173, 92.872586)","Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, 660000, Россия",О себе:\nдипломированный слесарь сантехник 4 разряд водитель ВС\n\nВозраст:\n36 лет\n\nГотовность к командировкам:\nИногда\n\nГражданство:\nРоссия


In [61]:
df_full["address_short"] = df_full["address"].apply(shorten_address)

In [62]:
def merge_edu(edu: str, school: str):
    if edu == school:
        return edu
    return f"{edu}, {school}".replace(", Нет данных", "").replace("Нет данных, ", "")

In [63]:
df_full["Образование"] = df_full[["Образование", "Учебные заведения"]].apply(lambda x: merge_edu(x[0], x[1]), axis = 1)
df_full = df_full.drop(columns=["Учебные заведения"])

/var/folders/1k/4qnh82lj5f16195540wwdb5h0000gn/T/ipykernel_469/3358831164.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_full["Образование"] = df_full[["Образование", "Учебные заведения"]].apply(lambda x: merge_edu(x[0], x[1]), axis = 1)


In [64]:
df_full.head(2)

,link,address,Тип занятости,Доступность,Образование,Переезд,Компетенции,График работы,Опыт работы,Категория прав,id,itemDate,Должность,Дата осмотра терапевтом,address_short,coords,address_full,Описание
153,/baykit/rezume/slesar_santehnik_7259471769?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJYc3VwMnpSeEE5MWE5dmhyIjt9R4egVz8AAAA,"Красноярский край, Эвенкийский район",Основная работа,Да,Среднее,Невозможен,"ЖКХ, эксплуатация",Полный день,"Стаж работы:\n7 лет\n\nОпыт работы:\nМастер учястка, Проводил монтаж работы, Апрель 2018 — настоящее время, ООО ЭЛ-Техника, 7 лет и 1 месяц",Водительское удостоверение:\nНет данных\n\nКатегория прав:\nНет данных\n\nСтаж вождения:\nНет данных,№ 7327731583,· 21 апреля в 14:23,Слесарь-сантехник,Нет данных,"Красноярский край, Эвенкийский район","(64.2695782, 99.5609121)","Эвенкийский район, Красноярский край, Сибирский федеральный округ, Россия","О себе:\nспокойствие, обучаемость,\n\nВозраст:\n35 лет\n\nГотовность к командировкам:\nНе готов\n\nГражданство:\nРоссия"
158,/krasnoyarsk/rezume/slesar_santehnik_7283925077?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJYc3VwMnpSeEE5MWE5dmhyIjt9R4egVz8AAAA,"Красноярский край, Красноярск, ул. Березина, 82В\nр-н Центральный","Основная работа, Подработка",Да,"Высшее, Природообустройство и водопользование, брянский государственный аграрный университет",Невозможен,Строительство,Сменный график,"Стаж работы:\n4 года\n\nОпыт работы:\nИюль 2022 — Ноябрь 2023, 1 год и 5 месяцев, МУП ""Отрадное"", Слесарь сантехник, Сантехнические работы",Водительское удостоверение:\nНет данных\n\nКатегория прав:\nНет данных\n\nСтаж вождения:\nНет данных,№ 7323042571,· 19 апреля в 20:54,Слесарь-сантехник,Нет данных,"Красноярский край, Красноярск","(56.0091173, 92.872586)","Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, 660000, Россия",О себе:\nдипломированный слесарь сантехник 4 разряд водитель ВС\n\nВозраст:\n36 лет\n\nГотовность к командировкам:\nИногда\n\nГражданство:\nРоссия


In [65]:
months_dict = {
    "января": "01",
    "февраля": "02",
    "марта": "03",
    "апреля": "04",
    "мая": "05",
    "июня": "06",
    "июля": "07",
    "августа": "08",
    "сентября": "09",
    "октября": "10",
    "ноября": "11",
    "декабря": "12"
}

In [66]:
def prep_date(date: str):
    date_ = date.replace("· ", "").replace("сегодня", "28 января").replace("вчера", "27 января").strip()
    date_ = re.sub(r'\s+', ' ', date_)
    split_ = list(date_.split(" "))
    if " в " in date_:
        split_ = split_[:2]
        split_.append("2025")
    try:
        split_[-2] = months_dict[split_[-2]]
    except:
        print(split_)
        return float("nan")
    date_ = ".".join(split_)
    return date_

In [67]:
prep_date(df_full["itemDate"].iloc[0])

'21.04.2025'

In [68]:
df_full["date"] = df_full["itemDate"].apply(prep_date)

['Нет', 'данных']
['Нет', 'данных']
['Нет', 'данных']


In [69]:
df_full["date"]

153     21.04.2025
158     19.04.2025
161     22.04.2025
162      8.04.2025
163     27.01.2025
           ...    
2130    23.04.2025
2131    17.04.2025
2134    28.01.2025
2136    11.04.2025
2139    30.03.2025
Name: date, Length: 2373, dtype: object

In [70]:
df_full["date"] = pd.to_datetime(df_full["date"], dayfirst=True)
df_full = df_full.drop(columns=["itemDate"])

In [71]:
df_full = df_full.dropna(subset = ["date"])

In [72]:
df_full.head(2)

,link,address,Тип занятости,Доступность,Образование,Переезд,Компетенции,График работы,Опыт работы,Категория прав,id,Должность,Дата осмотра терапевтом,address_short,coords,address_full,Описание,date
153,/baykit/rezume/slesar_santehnik_7259471769?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJYc3VwMnpSeEE5MWE5dmhyIjt9R4egVz8AAAA,"Красноярский край, Эвенкийский район",Основная работа,Да,Среднее,Невозможен,"ЖКХ, эксплуатация",Полный день,"Стаж работы:\n7 лет\n\nОпыт работы:\nМастер учястка, Проводил монтаж работы, Апрель 2018 — настоящее время, ООО ЭЛ-Техника, 7 лет и 1 месяц",Водительское удостоверение:\nНет данных\n\nКатегория прав:\nНет данных\n\nСтаж вождения:\nНет данных,№ 7327731583,Слесарь-сантехник,Нет данных,"Красноярский край, Эвенкийский район","(64.2695782, 99.5609121)","Эвенкийский район, Красноярский край, Сибирский федеральный округ, Россия","О себе:\nспокойствие, обучаемость,\n\nВозраст:\n35 лет\n\nГотовность к командировкам:\nНе готов\n\nГражданство:\nРоссия",2025-04-21
158,/krasnoyarsk/rezume/slesar_santehnik_7283925077?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJYc3VwMnpSeEE5MWE5dmhyIjt9R4egVz8AAAA,"Красноярский край, Красноярск, ул. Березина, 82В\nр-н Центральный","Основная работа, Подработка",Да,"Высшее, Природообустройство и водопользование, брянский государственный аграрный университет",Невозможен,Строительство,Сменный график,"Стаж работы:\n4 года\n\nОпыт работы:\nИюль 2022 — Ноябрь 2023, 1 год и 5 месяцев, МУП ""Отрадное"", Слесарь сантехник, Сантехнические работы",Водительское удостоверение:\nНет данных\n\nКатегория прав:\nНет данных\n\nСтаж вождения:\nНет данных,№ 7323042571,Слесарь-сантехник,Нет данных,"Красноярский край, Красноярск","(56.0091173, 92.872586)","Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, 660000, Россия",О себе:\nдипломированный слесарь сантехник 4 разряд водитель ВС\n\nВозраст:\n36 лет\n\nГотовность к командировкам:\nИногда\n\nГражданство:\nРоссия,2025-04-19


In [74]:
df_full["Доступность"].value_counts()

Доступность
Нет данных    1130
Да             812
Нет            428
Name: count, dtype: int64

In [76]:
df_full["Должность"].value_counts()

Должность
Сварщик              1108
Электромонтажник      703
Инженер-энергетик     453
Слесарь-сантехник     106
Name: count, dtype: int64

In [78]:
df_full = df_full.rename(columns = {"id": "ID"})

In [80]:
df_prev = pd.read_csv("../../uniweb-cv-match/data_mass/candidates.csv")

In [81]:
df_final = pd.concat([df_prev, df_full])

In [82]:
df_final.shape

(4270, 18)

In [83]:
df_final.to_csv("../../uniweb-cv-match/data_mass/candidates.csv", index=False)